In [1]:
# test.py
import pandas as pd
from ArticleTools import crawling_news

df = pd.DataFrame(crawling_news.get_req())
df

,title,points,link
0,Forbes Marketplace - 숙주를 먹어치우려는 기생 SEO 회사의 진실 ...,4,https://larslofgren.com/forbes-marketplace/
1,"EU, 아이폰에 IOS 타사 개방 요구 (news.naver.com)",3,https://news.naver.com/mnews/article/029/00029...
2,"GN⁺: 닌텐도, Palworld 개발사 PocketPair 상대로 특허권 침해 소...",1,https://www.nintendo.co.jp/corporate/release/e...
3,GN⁺: Scramble - Grammarly 대체 오픈소스 (github.com...,7,https://github.com/zlwaterfield/scramble
4,"Oracle, 이제 JavaScript를 놓아줘야 할 때입니다. (javascrip...",14,https://javascript.tm/
5,Safari 18.0의 WebKit 기능들 (webkit.org),4,https://webkit.org/blog/15865/webkit-features-...
6,GN⁺: 책을 쓰지 말아야 하는 이유 (gwern.net),12,https://gwern.net/book-writing
7,GN⁺: MrBeast의 프로덕션 성공 비법 (유출된 PDF) (simonwilli...,34,https://simonwillison.net/2024/Sep/15/how-to-s...
8,The HTTP Query Method (ietf.org),11,https://www.ietf.org/archive/id/draft-ietf-htt...
9,"애플, iOS 18 출시 (apple.com)",4,https://www.apple.com/kr/newsroom/2024/09/ios-...


In [2]:
df['link']

0           https://larslofgren.com/forbes-marketplace/
1     https://news.naver.com/mnews/article/029/00029...
2     https://www.nintendo.co.jp/corporate/release/e...
3              https://github.com/zlwaterfield/scramble
4                                https://javascript.tm/
5     https://webkit.org/blog/15865/webkit-features-...
6                        https://gwern.net/book-writing
7     https://simonwillison.net/2024/Sep/15/how-to-s...
8     https://www.ietf.org/archive/id/draft-ietf-htt...
9     https://www.apple.com/kr/newsroom/2024/09/ios-...
10    https://peterszasz.com/how-to-lead-your-team-w...
11          https://www.youtube.com/watch?v=j2VBKvvmKKM
12     https://www.matthewball.co/all/fansprofitandloss
13    https://www.reuters.com/technology/how-intel-l...
14    https://rethinkingsoftware.substack.com/p/why-...
15    https://timculpan.substack.com/p/apple-mobile-...
16       https://www.swift.org/blog/announcing-swift-6/
17                  https://news.hada.io/topic?i

In [3]:
from langchain.document_loaders import WebBaseLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

loader = WebBaseLoader(df['link'][13])
data = loader.load()

text_splitter=RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=0) 
all_splits = text_splitter.split_documents(data)

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [4]:
all_splits

[Document(metadata={'source': 'https://www.reuters.com/technology/how-intel-lost-sony-playstation-business-2024-09-16/', 'title': 'reuters.com', 'language': 'No language found.'}, page_content='reuters.comPlease enable JS and disable any ad blocker')]

In [5]:
from langchain_community.llms import Ollama
from langchain import PromptTemplate

model = Ollama(model='gemma2:2b', stop=["<|eot_id|>"]) # Added the stop token

In [6]:
user_prompt = "Summarize the content and explain in detail by korean."

system_prompt = """Think and write your step-by-step reasoning before responding.
Make sure you don't deviate from json grammar.
Please write all conversations in Korean(한국어).
"""

In [7]:
from langchain.vectorstores import Chroma
from langchain.embeddings import SentenceTransformerEmbeddings

embeddings = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")
vectorstore = Chroma.from_documents(documents=all_splits, embedding=embeddings)
docs = vectorstore.similarity_search(user_prompt)

/home/kar/Projects/News_Summary/venv/lib/python3.12/site-packages/pydantic/_internal/_fields.py:132: UserWarning: Field "model_name" in HuggingFaceInferenceAPIEmbeddings has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(
/tmp/ipykernel_2318/2986656654.py:4: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  embeddings = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")
/home/kar/Projects/News_Summary/venv/lib/python3.12/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See ht

In [8]:
# NOTE: No f-string and no whitespace in curly braces.
template = """
    <|begin_of_text|>
    <|start_header_id|>system<|end_header_id|>
    {system_prompt}
    <|eot_id|>
    <|start_header_id|>user<|end_header_id|>
    {user_prompt}
    <|eot_id|>
    <|start_header_id|>assistant<|end_header_id|>
"""

prompt = PromptTemplate(
    input_variables=['system_prompt', 'user_prompt'],
    template=template
)

from langchain.chains import RetrievalQA

qachain = RetrievalQA.from_chain_type(model, retriever=vectorstore.as_retriever())

response = qachain(prompt.format(system_prompt=system_prompt, user_prompt=user_prompt))

/tmp/ipykernel_2318/2901870035.py:22: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use invoke instead.
  response = qachain(prompt.format(system_prompt=system_prompt, user_prompt=user_prompt))
Number of requested results 4 is greater than number of elements in index 1, updating n_results = 1


In [9]:
response

{'query': "\n    <|begin_of_text|>\n    <|start_header_id|>system<|end_header_id|>\n    Think and write your step-by-step reasoning before responding.\nMake sure you don't deviate from json grammar.\nPlease write all conversations in Korean(한국어).\n\n    <|eot_id|>\n    <|start_header_id|>user<|end_header_id|>\n    Summarize the content and explain in detail by korean.\n    <|eot_id|>\n    <|start_header_id|>assistant<|end_header_id|>\n",
 'result': '죄송합니다만, 제공된 텍스트를 분석할 수 없습니다. 필요한 정보가 부족합니다.  영어로 된 질문을 입력하시면 최선을 다해 답변드릴 수 있습니다. \n'}

In [10]:
print(response['result'])

죄송합니다만, 제공된 텍스트를 분석할 수 없습니다. 필요한 정보가 부족합니다.  영어로 된 질문을 입력하시면 최선을 다해 답변드릴 수 있습니다. 

